# 1导入数据

导入库函数

In [ ]:
import numpy as np
import pandas as pd
import glob

In [ ]:
import numpy as np
import pandas as pd
import glob
labelDict = {'1': 0,'2':1,'3':2,'G11': 3,'G12':4,'G13':5}
folders = glob.glob('indata/npy224/*')
# 数据
datas = []
for i in range(6):
    datas.append([])
    pass

# 载入数据
for folder in folders:      
    folder = folder.replace('\\', '/')
    label = labelDict[folder.split('/')[-1].split('_')[0]]
    # 数据提取
    npys = glob.glob(folder + '/*.npy')
    for npy in npys:
        npy = npy.replace('\\', '/')
        # data_npy = np.load(file=npy)
        datas[label].append(npy)    
        pass
    pass
datas
datanp = np.array(datas)
datanp.shape

划分训练集合测试集

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# datanp=shuffle(datanp) 
train_data = []
train_label = []
test_data = []
test_label = []

for label in range(6):
    traindata,testdata = train_test_split(datanp[label],test_size=0.2,random_state=0,shuffle=True)
    train_data.extend(traindata)
    test_data.extend(testdata)
    train_shape = np.array(traindata).shape
    test_shape = np.array(testdata).shape
    train_label.extend(np.full(train_shape,label))
    test_label.extend(np.full(test_shape,label))
     
# train_data = np.array(train_data)
# train_label = np.array(pd.get_dummies(train_label))
# test_data = np.array(test_data)
# test_label = np.array(pd.get_dummies(test_label))

train_pd = pd.DataFrame(columns=['datapath','label'])
test_pd = pd.DataFrame(columns=['datapath','label'])
for i in range(0,len(train_data)):
    train_pd = train_pd._append(pd.Series({'datapath':train_data[i],'label':train_label[i]}),ignore_index=True)
    pass
for i in range(0,len(test_data)):
    test_pd = test_pd._append(pd.Series({'datapath':test_data[i],'label':test_label[i]}),ignore_index=True)
    pass
train_pd = shuffle(train_pd)
test_pd = shuffle(test_pd)
# train_pd = pd.concat([pd.DataFrame(train_data),pd.DataFrame(train_label)],axis=1)
# test_pd = pd.concat([pd.DataFrame(test_data),pd.DataFrame(test_label)],axis=1)
train_data = np.array(train_pd['datapath'])
train_label = np.array(pd.get_dummies(train_pd['label']))
test_data = np.array(test_pd['datapath'])
test_label = np.array(pd.get_dummies(test_pd['label']))
# test_label = np.array(test_label)
# 训练模型
print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)



导入之前的训练集，测试集结果

In [ ]:
train_data=np.load("train_data.npy",allow_pickle=True)
test_data=np.load("test_data.npy",allow_pickle=True)
train_label=np.load("train_label.npy",allow_pickle=True)
test_label=np.load("test_label.npy",allow_pickle=True)

数据拼接

In [ ]:
inputs = np.concatenate((train_data, test_data), axis=0)
targets = np.concatenate((train_label, test_label), axis=0)

# 2建模

In [ ]:
import tensorflow as tf
from tensorflow.python import keras
import keras
import keras.callbacks
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

 

def get_model2D(width=30, height=30, channel=224):
    """Build a 2D convolutional neural network model."""
    inputs = keras.Input((width, height, channel))
    # aver=tf.reduce_mean(inputs,axis=0)
    # aver1=tf.reduce_mean(aver,axis=1)
    # aver1=tf.reduce_mean(aver1,axis=0)
    # print(aver1.shape)
    # aver2=layers.Dense(units=54)(aver1)
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1,kernel_regularizer=regularizers.L1(0.003))(inputs)  #激活函数要不要改成热炉
    x= layers.Conv2D( filters=64,kernel_size=3, strides=1,kernel_regularizer=regularizers.L2(0.003),activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=64,kernel_initializer=keras.initializers.glorot_uniform(seed=None))(x)  #图像 模型得到
    x = layers.LayerNormalization()(x)
    outputs = layers.Dense(units=6,activation='softmax')(x)
    # Define the model.
    model = keras.Model(inputs, outputs, name="2dcnn")
    return model
# Build model.
model = get_model2D(width=30, height=30, channel=224)
model.summary()

# 3训练前

归一化/批处理

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def get_train_batch_3D(x_train_paths, y_labels, batch_size):
    '''
    参数：
        X_train：所有图片路径列表
        y_train: 所有图片对应的标签列表
        batch_size:批次
        img_w:图片宽
        img_h:图片高
        color_type:图片类型
        is_argumentation:是否需要数据增强
    返回: 
        一个generator，x: 获取的批次图片 y: 获取的图片对应的标签
    '''
    while 1:
        for i in range(0,len(x_train_paths),batch_size):
            x_train = np.load(file=x_train_paths[i])
            y_label = y_labels[i]

            x_train[np.isinf(x_train)] = 0
            x_train[np.isnan(x_train)] = 0

            # x_train = np.expand_dims(x_train,axis=0)
            # y_label = np.expand_dims(y_label,axis=0)           
            # print(x_train.shape)
            # print(y_label.shape)
            yield(np.array(x_train), np.array(y_label))

def get_train_batch_2D(x_train_paths, y_labels, batch_size):
    '''
    参数：
        X_train：所有图片路径列表
        y_train: 所有图片对应的标签列表
        batch_size:批次
        img_w:图片宽
        img_h:图片高
        color_type:图片类型
        is_argumentation:是否需要数据增强
    返回: 
        一个generator，x: 获取的批次图片 y: 获取的图片对应的标签
    '''
    while 1:
        for i in range(0,len(x_train_paths),batch_size):
            x_train = np.load(file=x_train_paths[i])
            x_train = scaler.fit_transform(
                 x_train.astype(np.float32).reshape(-1,1)).reshape(30, 30,224)
            y_label = y_labels[i]
            x_train[np.isinf(x_train)] = 0
            x_train[np.isnan(x_train)] = 0
            x_train = np.expand_dims(x_train,axis=0)
            y_label = np.expand_dims(y_label,axis=0)
            yield(np.array(x_train), np.array(y_label)) 
    print("finish") 


固定随机种子

In [ ]:

import random
import numpy as np
import tensorflow as tf

# 固定 Python 随机种子
random.seed(42)

# 固定 Numpy 随机种子
np.random.seed(42)

# 固定 TensorFlow 随机种子
tf.random.set_seed(42)

# 4模型训练

导入模型

In [ ]:
model = keras.models.load_model("425new-0.9-0.88")

导入weights

In [ ]:
model.load_weights(r'model_save\84-90best511.h5 ') 

开始训练

In [ ]:
# 第一个，现在的模型有8层左右
# 第一个，现在的模型有8层左右
import keras.optimizers
import keras.losses
import keras.metrics
import keras.callbacks
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
# lr=1e-3,decay=1e-6,momentum=0.9     lr=1e-3,decay=1e-6,momentum=0.9,nesterov=True, clipnorm=1.
'''用不同的优化器训练模型'''
rms=optimizers.rmsprop_v2.RMSProp(lr=0.0000015) 
sgd2=optimizers.gradient_descent_v2.SGD(lr=0.0000001,momentum=0.1,nesterov=True)
adelta=optimizers.adadelta_v2.Adadelta(lr=0.00001)
adam=keras.optimizers.adam_v2.Adam(lr=0.000000001)

'''自动保存最优模型'''
checkpoint = keras.callbacks.ModelCheckpoint('./model_save/best.h5', monitor='val_acc', verbose=0,
                                save_best_only=True,
                                save_weights_only=False, mode='max',
                                period=1)
'''设置早停'''
earlyStop = EarlyStopping(monitor='val_acc', min_delta=0.003, patience=20, mode='max', verbose=1, restore_best_weights = True)

# K-fold Cross Validation model 
'''设置交叉验证的次数'''
num_folds=2

kfold = KFold(n_splits=num_folds, shuffle=True)
model.compile(
optimizer=sgd2, 
loss= keras.losses.CategoricalCrossentropy(),
metrics=['acc'])
for train, test in kfold.split(inputs, targets):
    history = model.fit_generator(generator=get_train_batch_2D(inputs[train],targets[train],8), 
        steps_per_epoch=len(train_data), 
        epochs=200,
        shuffle=True,
        validation_data=get_train_batch_2D(inputs[test],targets[test],16),  
        validation_steps=len(test_data),
        callbacks=[checkpoint,keras.callbacks.TensorBoard(log_dir="./log/2D2",histogram_freq=1,write_graph=True,write_images=True,update_freq='batch')]
        ) 



训练过程画图

In [ ]:
import matplotlib.pyplot as plt
# 绘制训练 & 验证的准确率值
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 5保存内容

保存训练集测试集划分的结果

In [ ]:
train_data=np.save(train_data)
train_data=np.save(test_data)
train_data=np.save(train_label)
train_data=np.save(test_label)

保存训练模型

In [ ]:
model.save("425new-0.9-0.88")